In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import timedelta

In [2]:
df = pd.read_csv("MakerSpace-Snapshots-2022-06-03.csv")
df["timestamp_start"] = pd.to_datetime(df["timestamp_start"])
df['timestamp_end'] = pd.to_datetime(df['timestamp_end'])
df['timestamp_prod_start'] = pd.to_datetime(df['timestamp_prod_start'])
df['timestamp_prod_end'] = pd.to_datetime(df['timestamp_prod_end'])
df['timestamp_qual_start'] = pd.to_datetime(df['timestamp_qual_start'])
df['timestamp_qual_end'] = pd.to_datetime(df['timestamp_qual_end'])

In [3]:
right_format_data = df.iloc[-4 : , :].reset_index(drop = True)
right_format_data

,uuid,timestamp_start,env_iaq_index,env_iaq_accuracy,env_pressure,env_temperature,env_humidity,timestamp_prod_start,prod_obj_temp,prod_amb_temp,prod_plt_temp,timestamp_prod_end,timestamp_qual_start,qual_obj_temp,qual_amb_temp,qual_weight,qual_color,qual_height,timestamp_qual_end,timestamp_end
0,08537450-510f-4b99-b552-49d263e6f611,2022-06-02 09:10:29.639074,174,3,996.82,25.56,30.02,2022-06-02 09:10:36.891143,23.7,25.1,-1025.99,2022-06-02 09:10:46.941441,2022-06-02 09:10:56.828854,23.5,23.5,94.0,NaN,NaN,2022-06-02 09:11:06.932722,2022-06-02 09:11:32.302483
1,3f86f547-5489-45cc-b4f1-e3735b34c40e,2022-06-02 09:12:07.578195,175,3,996.81,25.57,29.88,2022-06-02 09:12:14.830009,23.8,25.1,-1025.99,2022-06-02 09:12:24.885306,2022-06-02 09:12:34.767733,23.4,23.4,94.0,NaN,NaN,2022-06-02 09:12:44.899782,2022-06-02 09:13:10.241268
2,fa720546-75dd-4af3-a680-023884a45edc,2022-06-02 09:14:37.566409,177,3,996.86,25.59,29.88,2022-06-02 09:14:44.818175,23.6,24.6,-1025.99,2022-06-02 09:14:54.880311,2022-06-02 09:15:04.755793,23.6,23.6,94.0,NaN,NaN,2022-06-02 09:15:14.877606,2022-06-02 09:15:40.229497
3,2fca757b-9977-4600-a519-fe15a061d5ea,2022-06-02 09:16:15.519023,175,3,996.84,25.61,29.82,2022-06-02 09:16:22.771024,23.7,24.7,-1025.99,2022-06-02 09:16:32.802121,2022-06-02 09:16:42.708717,23.8,23.8,94.0,NaN,NaN,2022-06-02 09:16:52.834750,2022-06-02 09:17:18.182453


In [4]:
ps = []
pe = []
qs = []
qe = []
te = []

for index, row in right_format_data.iterrows():
    if index == 0:
        prev_s = row["timestamp_start"]
        prev_e = row['timestamp_end']
        prev_prod_s = row['timestamp_prod_start']
        prev_prod_e = row['timestamp_prod_end']
        prev_qual_s = row['timestamp_qual_start']
        prev_qual_e = row['timestamp_qual_end']
    
    ps.append((row["timestamp_prod_start"] - row["timestamp_start"]).total_seconds())
    pe.append((row['timestamp_prod_end'] - row["timestamp_prod_start"]).total_seconds())
    qs.append((row['timestamp_qual_start'] - row['timestamp_prod_end']).total_seconds())
    qe.append((row['timestamp_qual_end'] - row['timestamp_qual_start']).total_seconds())
    te.append((row['timestamp_end'] - row['timestamp_qual_end']).total_seconds())
    
    print((row["timestamp_prod_start"] - row["timestamp_start"]).total_seconds())
    print((row['timestamp_prod_end'] - row["timestamp_prod_start"]).total_seconds())
    print((row['timestamp_qual_start'] - row['timestamp_prod_end']).total_seconds())
    print((row['timestamp_qual_end'] - row['timestamp_qual_start']).total_seconds())
    print((row['timestamp_end'] - row['timestamp_qual_end']).total_seconds())
    print()

7.252069
10.050298
9.887413
10.103868
25.369761

7.251814
10.055297
9.882427
10.132049
25.341486

7.251766
10.062136
9.875482
10.121813
25.351891

7.252001
10.031097
9.906596
10.126033
25.347703



In [5]:
dif_ps, dif_pe, dif_qs, dif_qe, dif_te = (np.mean(ps), np.mean(pe), np.mean(qs), np.mean(qe), np.mean(te))

In [6]:
timedelta(dif_ps)

datetime.timedelta(days=7, seconds=21765, microseconds=240000)

In [7]:
corrected_data_list = []

for index, row in df.fillna(0).iterrows():
    temp_dct = {}
    for key, value in row.to_dict().items():
        if ("timestamp" in key) and value == 0:
            if key == "timestamp_prod_start":
                temp_dct[key] = temp_dct['timestamp_start'] + timedelta(seconds = dif_ps)
            elif key == "timestamp_prod_end":
                temp_dct[key] = temp_dct['timestamp_prod_start'] + timedelta(seconds = dif_pe)
            elif key == "timestamp_qual_start":
                temp_dct[key] = temp_dct['timestamp_prod_end'] + timedelta(seconds = dif_qs)
            elif key == "timestamp_qual_end":
                temp_dct[key] = temp_dct['timestamp_qual_start'] + timedelta(seconds = dif_qe)
            elif key == "timestamp_end":
                temp_dct[key] = temp_dct['timestamp_qual_end'] + timedelta(seconds = dif_te)
        else:
            temp_dct[key] = value
    corrected_data_list.append(temp_dct)

corrected_data = pd.DataFrame(corrected_data_list)
corrected_data

,uuid,timestamp_start,env_iaq_index,env_iaq_accuracy,env_pressure,env_temperature,env_humidity,timestamp_prod_start,prod_obj_temp,prod_amb_temp,prod_plt_temp,timestamp_prod_end,timestamp_qual_start,qual_obj_temp,qual_amb_temp,qual_weight,qual_color,qual_height,timestamp_qual_end,timestamp_end
0,54f145d2-e871-4bb9-8f9f-ee72e2ced9cd,2022-01-27 09:15:54.499085,92,3,1006.69,20.89,28.28,2022-01-27 09:16:01.750997,18.4,19.8,18.54,2022-01-27 09:16:11.800704,2022-01-27 09:16:21.688684,19.0,19.0,304.0,0.0,0.0,2022-01-27 09:16:31.809625,2022-01-27 09:16:57.162335
1,a3ec2e1f-fa73-4c7c-9b2b-76c11e607ded,2022-01-27 09:17:32.314592,92,3,1006.72,20.92,28.28,2022-01-27 09:17:39.566504,19.1,19.9,33.32,2022-01-27 09:17:49.616211,2022-01-27 09:17:59.504191,20.1,20.1,304.0,0.0,0.0,2022-01-27 09:18:09.625132,2022-01-27 09:18:34.977842
2,3fad8ca6-c3f5-4646-8d13-4d12c23bce4b,2022-01-27 09:19:40.265449,89,3,1006.71,20.95,28.12,2022-01-27 09:19:47.517361,20.0,20.0,66.63,2022-01-27 09:19:57.567068,2022-01-27 09:20:07.455048,20.2,20.2,417.0,0.0,0.0,2022-01-27 09:20:17.575989,2022-01-27 09:20:42.928699
3,c047f085-5295-487c-983b-822973db31d8,2022-01-27 09:21:17.686957,88,0,1006.71,20.96,28.13,2022-01-27 09:21:24.938869,20.1,20.1,69.02,2022-01-27 09:21:34.988576,2022-01-27 09:21:44.876556,18.9,18.9,271.0,0.0,0.0,2022-01-27 09:21:54.997497,2022-01-27 09:22:20.350207
4,d635cbe2-f5dc-4bc2-af97-b7dfb4f52dbc,2022-01-27 09:22:55.582707,88,0,1006.69,21.00,28.08,2022-01-27 09:23:02.834619,21.5,20.3,70.03,2022-01-27 09:23:12.884326,2022-01-27 09:23:22.772306,20.0,20.0,536.0,0.0,0.0,2022-01-27 09:23:32.893247,2022-01-27 09:23:58.245957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,aa0d9200-5c90-4d3d-adb4-27045c6855c9,2022-06-02 09:05:29.429629,181,0,996.88,25.50,30.09,2022-06-02 09:05:36.681542,23.6,25.1,-1025.99,2022-06-02 09:05:46.777980,2022-06-02 09:05:56.619451,0.0,0.0,0.0,0.0,0.0,2022-06-02 09:06:06.740392,2022-06-02 09:06:32.093102
374,08537450-510f-4b99-b552-49d263e6f611,2022-06-02 09:10:29.639074,174,3,996.82,25.56,30.02,2022-06-02 09:10:36.891143,23.7,25.1,-1025.99,2022-06-02 09:10:46.941441,2022-06-02 09:10:56.828854,23.5,23.5,94.0,0.0,0.0,2022-06-02 09:11:06.932722,2022-06-02 09:11:32.302483
375,3f86f547-5489-45cc-b4f1-e3735b34c40e,2022-06-02 09:12:07.578195,175,3,996.81,25.57,29.88,2022-06-02 09:12:14.830009,23.8,25.1,-1025.99,2022-06-02 09:12:24.885306,2022-06-02 09:12:34.767733,23.4,23.4,94.0,0.0,0.0,2022-06-02 09:12:44.899782,2022-06-02 09:13:10.241268
376,fa720546-75dd-4af3-a680-023884a45edc,2022-06-02 09:14:37.566409,177,3,996.86,25.59,29.88,2022-06-02 09:14:44.818175,23.6,24.6,-1025.99,2022-06-02 09:14:54.880311,2022-06-02 09:15:04.755793,23.6,23.6,94.0,0.0,0.0,2022-06-02 09:15:14.877606,2022-06-02 09:15:40.229497


In [8]:
corrected_data.to_csv("corrected_old_data.csv")

In [27]:
ir_temp_heiz_df = corrected_data[['uuid', 'timestamp_prod_start', 'timestamp_start', 'prod_obj_temp', 'prod_amb_temp', 'timestamp_prod_end', 'timestamp_end']]
ir_temp_qual_df = corrected_data[['uuid', 'timestamp_qual_start', 'timestamp_start', 'qual_obj_temp', 'qual_amb_temp', 'timestamp_qual_end', 'timestamp_end']]
kamera_df = corrected_data[['uuid', 'timestamp_qual_start', 'qual_height', 'qual_color']]
thermo_couple_heiz_df = corrected_data[['uuid', 'timestamp_prod_start', 'prod_plt_temp', 'timestamp_prod_end']]
umgebung_df = corrected_data[['uuid', 'timestamp_start', 'env_temperature', 'env_humidity', 'env_iaq_index', 'env_pressure']]
waage_df = corrected_data[['uuid', 'timestamp_qual_start', 'qual_weight']]

In [31]:
ir_temp_heiz_df['start_date'] = ir_temp_heiz_df['timestamp_start'].apply(lambda x: x.date())
ir_temp_heiz_df['end_date'] = ir_temp_heiz_df['timestamp_end'].apply(lambda x: x.date())

ir_temp_qual_df['start_date'] = ir_temp_qual_df['timestamp_start'].apply(lambda x: x.date())
ir_temp_qual_df['end_date'] = ir_temp_qual_df['timestamp_end'].apply(lambda x: x.date())

C:\Users\mrnor\AppData\Local\Temp\ipykernel_13976\90403229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ir_temp_heiz_df['start_date'] = ir_temp_heiz_df['timestamp_start'].apply(lambda x: x.date())
C:\Users\mrnor\AppData\Local\Temp\ipykernel_13976\90403229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ir_temp_heiz_df['end_date'] = ir_temp_heiz_df['timestamp_end'].apply(lambda x: x.date())
C:\Users\mrnor\AppData\Local\Temp\ipykernel_13976\90403229.py:4: SettingWithCopyWarning: 
A value is trying 

In [33]:
ir_temp_heiz_df.to_csv("ir_temp_heiz_df.csv")
ir_temp_qual_df.to_csv("ir_temp_qual_df.csv")
kamera_df.to_csv("kamera_df.csv")
thermo_couple_heiz_df.to_csv("thermo_couple_heiz_df.csv")
umgebung_df.to_csv("umgebung_df.csv")
waage_df.to_csv("waage_df.csv")